## Smokes/Friends/Cancer with Logic Tensor Networks

This is a classic example from relational learning implemented in LTN and grounded in a domain $\mathbb{R}^n$.

In [3]:
import logging;logging.basicConfig(level=logging.INFO)
import tensorflow as tf
import logictensornetworks as ltn
import logictensornetworks_wrapper as ltnw

Some initial settings. Includes the dimensionality of the embedding/grounding space

In [22]:
ltn.LAYERS = 4
ltn.BIAS_factor = 1e-3

max_epochs=1000
embedding_size = 10 # embedding space dimensionality

We use three predicates. Two unary predicates and one binary predicate

In [23]:
ltnw.predicate('Smokes',embedding_size);
ltnw.predicate('Friends',embedding_size*2);
ltnw.predicate('Cancer',embedding_size);

Generate constants a to n

In [9]:
g1,g2='abcdefgh','ijklmn'
g=g1+g2
for l in g:
    ltnw.constant(l,min_value=[0.]*embedding_size,max_value=[1.]*embedding_size)

First we define the facts known about the world, i.e. who we know is friends, smokes and has cancer

In [11]:
friends = [('a','b'),('a','e'),('a','f'),('a','g'),('b','c'),('c','d'),('e','f'),('g','h'),
           ('i','j'),('j','m'),('k','l'),('m','n')]
[ltnw.formula("Friends(%s,%s)" %(x,y)) for (x,y) in friends]
[ltnw.formula("~Friends(%s,%s)" %(x,y)) for x in g1 for y in g1 if (x,y) not in friends and x < y]
[ltnw.formula("~Friends(%s,%s)" %(x,y)) for x in g2 for y in g2 if (x,y) not in friends and x < y]

smokes = ['a','e','f','g','j','n']
[ltnw.formula("Smokes(%s)" % x) for x in smokes]
[ltnw.formula("~Smokes(%s)" % x) for x in g if x not in smokes]

cancer = ['a','e']
[ltnw.formula("Cancer(%s)" % x) for x in cancer]
[ltnw.formula("~Cancer(%s)" % x) for x in g if x not in cancer]

print("\n".join(sorted(ltnw.FORMULAS.keys())))

Cancer(a)
Cancer(e)
Friends(a,b)
Friends(a,e)
Friends(a,f)
Friends(a,g)
Friends(b,c)
Friends(c,d)
Friends(e,f)
Friends(g,h)
Friends(i,j)
Friends(j,m)
Friends(k,l)
Friends(m,n)
Smokes(a)
Smokes(e)
Smokes(f)
Smokes(g)
Smokes(j)
Smokes(n)
~Cancer(b)
~Cancer(c)
~Cancer(d)
~Cancer(f)
~Cancer(g)
~Cancer(h)
~Cancer(i)
~Cancer(j)
~Cancer(k)
~Cancer(l)
~Cancer(m)
~Cancer(n)
~Friends(a,c)
~Friends(a,d)
~Friends(a,h)
~Friends(b,d)
~Friends(b,e)
~Friends(b,f)
~Friends(b,g)
~Friends(b,h)
~Friends(c,e)
~Friends(c,f)
~Friends(c,g)
~Friends(c,h)
~Friends(d,e)
~Friends(d,f)
~Friends(d,g)
~Friends(d,h)
~Friends(e,g)
~Friends(e,h)
~Friends(f,g)
~Friends(f,h)
~Friends(i,k)
~Friends(i,l)
~Friends(i,m)
~Friends(i,n)
~Friends(j,k)
~Friends(j,l)
~Friends(j,n)
~Friends(k,m)
~Friends(k,n)
~Friends(l,m)
~Friends(l,n)
~Smokes(b)
~Smokes(c)
~Smokes(d)
~Smokes(h)
~Smokes(i)
~Smokes(k)
~Smokes(l)
~Smokes(m)


Then we add knowledge that we know holds for each constant/individual. For this we use two variables, that go over the all samples

ltnw.variable("p",tf.concat(list(ltnw.CONSTANTS.values()),axis=0))
ltnw.variable("q",tf.concat(list(ltnw.CONSTANTS.values()),axis=0))

ltnw.formula("forall p: ~Friends(p,p)")
ltnw.formula("forall p,q:Friends(p,q) -> Friends(q,p)")
ltnw.formula("forall p: exists q: Friends(p,q)")
ltnw.formula("forall p,q:Friends(p,q) -> (Smokes(p)->Smokes(q))")
ltnw.formula("forall p: Smokes(p) -> Cancer(p)")
print("\n".join(sorted(filter(lambda x: x.startswith("forall"), ltnw.FORMULAS.keys()))))

Initialize the knowledgebase and optimize

In [18]:
optimizer=tf.train.RMSPropOptimizer(learning_rate=.01,decay=.9)
formula_aggregator=lambda *x: tf.reduce_mean(tf.concat(x,axis=0))+ltn.BIAS
ltnw.initialize_knowledgebase(initial_sat_level_threshold=.1,optimizer=optimizer,formula_aggregator=formula_aggregator)
ltnw.train(track_sat_levels=1000,sat_level_epsilon=.99,max_epochs=max_epochs);

INFO:logictensornetworks_wrapper:Initializing knowledgebase
INFO:logictensornetworks_wrapper:Initializing optimizer
INFO:logictensornetworks_wrapper:Assembling feed dict
INFO:logictensornetworks_wrapper:Initializing Tensorflow session
INFO:logictensornetworks_wrapper:INITIALIZED with sat level = 0.4856998
INFO:logictensornetworks_wrapper:TRAINING 0 sat level -----> 0.4856998
INFO:logictensornetworks_wrapper:TRAINING finished after 999 epochs with sat level 0.95505595


In [ ]:
Check some formulas

In [20]:
for x in g:
    print(x," = ",ltnw.ask(x).squeeze())
    print("Cancer("+x+"): %.2f" % ltnw.ask("Cancer(%s)" % x).squeeze())
    print("Smokes("+x+"): %.2f" % ltnw.ask("Smokes(%s)" % x).squeeze())

    for y in g:
        print("Friends("+x+","+y+"): %.2f" % ltnw.ask("Friends(%s,%s)" % (x,y)).squeeze())


a  =  [-0.12019304 -0.29417938  0.4367437   1.1439954   0.68295485  0.02202765
  0.05832971 -0.26626703  0.8555932   1.6106366 ]
Cancer(a): 1.00
Smokes(a): 1.00
Friends(a,a): 0.00
Friends(a,b): 1.00
Friends(a,c): 0.00
Friends(a,d): 0.00
Friends(a,e): 1.00
Friends(a,f): 1.00
Friends(a,g): 0.95
Friends(a,h): 0.00
Friends(a,i): 0.95
Friends(a,j): 1.00
Friends(a,k): 1.00
Friends(a,l): 0.95
Friends(a,m): 0.00
Friends(a,n): 1.00
b  =  [-0.4005215   0.83488595 -0.28701967  1.3484085  -0.65651083  0.74934745
  1.4064897   0.53272015  0.5980132   0.27026483]
Cancer(b): 0.00
Smokes(b): 0.00
Friends(b,a): 0.99
Friends(b,b): 0.00
Friends(b,c): 1.00
Friends(b,d): 0.00
Friends(b,e): 0.00
Friends(b,f): 0.00
Friends(b,g): 0.00
Friends(b,h): 0.00
Friends(b,i): 0.92
Friends(b,j): 0.95
Friends(b,k): 0.00
Friends(b,l): 0.00
Friends(b,m): 1.00
Friends(b,n): 0.00
c  =  [ 1.5567048  -0.5668615   0.24863623  0.5336552   0.5450335   0.83997875
  0.2084002   0.59342045  0.8495948   0.5454229 ]
Cancer(c): 0.00
S

Check some general axioms

In [21]:
for formula in ["forall p: ~Friends(p,p)",
                "forall p,q: Friends(p,q) -> Friends(q,p)",
                "forall p: exists q: Friends(p,q)",
                "forall p,q: Friends(p,q) -> (Smokes(p)->Smokes(q))"]:
    print(formula,": %.2f" % ltnw.ask(formula).squeeze())

forall p: ~Friends(p,p) : 1.00
forall p,q: Friends(p,q) -> Friends(q,p) : 0.76
forall p: exists q: Friends(p,q) : 0.99
forall p,q: Friends(p,q) -> (Smokes(p)->Smokes(q)) : 0.00
